In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/12/15 12:38:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Reading the data into a Dataframe

In [2]:
df =  spark.read.json("/home/ubuntu/iot_devices")
df.show()

+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|           cn|device_id|         device_name|humidity|             ip|latitude|   lcd|longitude|  scale|temp|    timestamp|
+-------------+---------+----+----+-------------+---------+--------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|            8|      868|  US| USA|United States|        1|meter-gauge-1xbYRYcj|      51|   68.161.225.1|    38.0| green|    -97.0|Celsius|  34|1458444054093|
|            7|     1473|  NO| NOR|       Norway|        2|   sensor-pad-2n2Pea|      70|  213.161.254.1|   62.47|   red|     6.15|Celsius|  11|1458444054119|
|            2|     1556|  IT| ITA|        Italy|        3| device-mac-36TWSKiT|      44|      88.36.5.1|   42.83|   red|    12.83|Celsius|  19|1458444054120|
|            6|     1080|  US| USA|United Stat

# 2. Convert the Dataframe into a temporary view called iot.

In [6]:
df.createOrReplaceTempView('iot')
spark.sql("select * from iot").show(10, False)

+-------------+---------+----+----+-------------+---------+---------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn           |device_id|device_name          |humidity|ip             |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-------------+---------+---------------------+--------+---------------+--------+------+---------+-------+----+-------------+
|8            |868      |US  |USA |United States|1        |meter-gauge-1xbYRYcj |51      |68.161.225.1   |38.0    |green |-97.0    |Celsius|34  |1458444054093|
|7            |1473     |NO  |NOR |Norway       |2        |sensor-pad-2n2Pea    |70      |213.161.254.1  |62.47   |red   |6.15     |Celsius|11  |1458444054119|
|2            |1556     |IT  |ITA |Italy        |3        |device-mac-36TWSKiT  |44      |88.36.5.1      |42.83   |red   |12.83    |Celsius|19  |1458444054120|
|6            |1080     |US  |USA |Unite

# 3. Count how many devices are there from each country and display the output.

In [40]:
spark.sql("select cn, count(device_id) from iot group by cn order by cn").show(30)

+--------------------+----------------+
|                  cn|count(device_id)|
+--------------------+----------------+
|                    |            1810|
|         Afghanistan|              19|
|             Albania|              32|
|             Algeria|              34|
|      American Samoa|               3|
|             Andorra|               4|
|              Angola|              28|
|            Anguilla|               7|
| Antigua and Barbuda|              60|
|           Argentina|             978|
|             Armenia|              34|
|               Aruba|               8|
|           Australia|            3119|
|             Austria|            1462|
|          Azerbaijan|              49|
|             Bahamas|              18|
|             Bahrain|              55|
|          Bangladesh|             153|
|            Barbados|              38|
|             Belarus|             123|
|             Belgium|             666|
|              Belize|              13|


# 4. Display all the countries whose carbon dioxide level is more than 1400. Sort the output in descending order.

In [41]:
spark.sql("select cn, max(c02_level) as max_c02_level from iot group by cn having max_c02_level > 1400 order by max_c02_level DESC").show()

+-----------------+-------------+
|               cn|max_c02_level|
+-----------------+-------------+
|          Finland|         1599|
|         Thailand|         1599|
|           Sweden|         1599|
|          Ireland|         1599|
|             Peru|         1599|
|      Philippines|         1599|
|    United States|         1599|
|           Turkey|         1599|
|            China|         1599|
|           Russia|         1599|
|            India|         1599|
|           France|         1599|
|          Bolivia|         1599|
|        Argentina|         1599|
|            Italy|         1599|
|Republic of Korea|         1599|
|           Norway|         1599|
|          Denmark|         1599|
|            Spain|         1599|
|          Germany|         1599|
+-----------------+-------------+
only showing top 20 rows



# 5. Select all countries' devices with high-levels of C02 and group by cca3 and order by device_ids (Hint: For high CO2 level, the LCD status will be RED).

In [47]:
spark.sql("select cca3, count(device_id) as high_c02_devices_count from iot where lcd='red' group by cca3  order by high_c02_devices_count").show()

+----+----------------------+
|cca3|high_c02_devices_count|
+----+----------------------+
| IOT|                     1|
| CIV|                     1|
| MHL|                     1|
| GIB|                     1|
| TLS|                     1|
| GGY|                     1|
| LBR|                     1|
| SLB|                     1|
| GAB|                     1|
| TJK|                     1|
| REU|                     1|
| FLK|                     1|
| FSM|                     1|
| TKM|                     1|
| AND|                     1|
| ABW|                     2|
| VUT|                     2|
| LCA|                     2|
| GMB|                     2|
| VCT|                     2|
+----+----------------------+
only showing top 20 rows



In [49]:
spark.sql("select cca3, device_id from iot where lcd='red' order by cca3, device_id").show()

+----+---------+
|cca3|device_id|
+----+---------+
| ABW|     3251|
| ABW|   196547|
| AFG|   122437|
| AFG|   131489|
| AFG|   136829|
| AFG|   172471|
| AFG|   191633|
| AFG|   197785|
| AGO|    24240|
| AGO|    30228|
| AGO|    33254|
| AGO|    40274|
| AGO|    65166|
| AGO|    86009|
| AGO|   166577|
| AGO|   187756|
| ALA|    37695|
| ALA|    64390|
| ALA|   104561|
| ALA|   162539|
+----+---------+
only showing top 20 rows



# 6. Find out all devices in countries whose batteries need replacements.

In [22]:
spark.sql("select distinct(battery_level) from iot").show()

+-------------+
|battery_level|
+-------------+
|            0|
|            7|
|            6|
|            9|
|            5|
|            1|
|            3|
|            8|
|            2|
|            4|
+-------------+



In [58]:
spark.sql("select cn, device_id, device_name, battery_level from iot where battery_level = 0").show()

+-----------------+---------+--------------------+-------------+
|               cn|device_id|         device_name|battery_level|
+-----------------+---------+--------------------+-------------+
|            Japan|        8|sensor-pad-8xUD6p...|            0|
|    United States|       12|sensor-pad-12Y2kIm0o|            0|
|    United States|       17|meter-gauge-17zb8...|            0|
|          Germany|       44| sensor-pad-448DeWGL|            0|
|           Canada|       80|sensor-pad-80TY4d...|            0|
|Republic of Korea|       92| sensor-pad-92vxuq7e|            0|
|    United States|       98|sensor-pad-98mJQA...|            0|
|            Japan|      107|meter-gauge-1075K...|            0|
|        Australia|      111|device-mac-111WYt...|            0|
|    United States|      116| sensor-pad-11663yUf|            0|
|            China|      117|device-mac-117mcc...|            0|
|    United States|      123|device-mac-123zvY...|            0|
|    United States|      